# 项目：整理Netflix电影演员评分数据

## 分析目标

此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

本实战项目的目的在于练习整理数据，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netflix电视剧及电影数据。数据集包含两个数据表：`titles.csv`和`credits.csv`。

`titles.csv`包含电影及电视剧相关信息，包括影视作品ID、标题、类型、描述、流派、IMDB（一个国外的在线评分网站）评分，等等。`credits.csv`包含超过7万名出现在Netflix影视作品的导演及演员信息，包括名字、影视作品ID、人物名、演职员类型（导演/演员）等。

`titles.csv`每列的含义如下：
- id：影视作品ID
- title：影视作品标题
- show_type：作品类型（电视节目或电影）
- description：简短描述
- release_year：发布年份
- age_certification：适龄认证
- runtime：每集电视剧或电影的长度
- genres：流派类型
- production_countries：出品国家
- seasons：如果是电视剧，则是季数
- imdb_id：IMDB的ID
- imdb_score：IMDB的评分
- imdb_votes：IMDB的投票数
- tmdb_popularity：TMDB的流行度
- tmdb_score：TMDB的评分

`credits.csv`每列的含义如下：
- person_ID：演职员ID
- id：参与的影视作品ID
- name：演员姓名
- character_name：角色姓名
- role：演职员类型（演员或导演）

**根据需求，针对相关的数据，id，genres，imdb_score，name等，进行分析评估，一些数据在内容上不影响需求分析，根据情况整理或忽略**

## **导入文件**

In [1]:
import pandas as pd

In [2]:
origina_titles = pd.read_csv('10_titles.csv')        #读取文件，解析为 DataFrame
original_credits = pd.read_csv('10_credits.csv')  

In [3]:
cleaned_titles = origina_titles.copy()              #备份文件，之后操作在备份文件上进行
cleaned_credits = original_credits.copy()

## **数据结构整齐度**

In [4]:
cleaned_titles.head()            #查看前五行信息，genres和 production_countries一个单元格包含多个值，应进行拆分

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [5]:
cleaned_credits.sample(5)      # 查看随机 5行

,person_id,id,name,character,role
26969,2020029,tm494288,Sabrina Stadler,NaN,ACTOR
54167,51574,tm838876,Leslie Small,NaN,DIRECTOR
54449,27410,tm467467,Elysandra Quinones,Dancer,ACTOR
18783,65977,tm145627,Arunoday Singh,NaN,ACTOR
74186,592793,tm1001095,Rajesh Jais,Roohi's Father,ACTOR


In [6]:
cleaned_titles['genres'][1]           # "['drama', 'crime']" 列表在引号内，为字符串而非列表，拆分需要先进行类型转换

"['drama', 'crime']"

In [7]:
cleaned_titles['production_countries'][1]      # 同样字符串结构

"['US']"

In [8]:
cleaned_titles['genres'] = cleaned_titles['genres'].apply(lambda s: eval(s))    # 通过匿名函数对字符串进行类型转换
cleaned_titles['genres'][1]

['drama', 'crime']

In [9]:
cleaned_titles = cleaned_titles.explode("genres")   # 类型转换之后用 explode把选定列的值拆分成并列的行
cleaned_titles.head(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,thriller,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,european,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,fantasy,['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,action,['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,comedy,['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811


In [10]:
cleaned_titles['production_countries'] = cleaned_titles['production_countries'].apply(lambda s: eval(s))    # 类型转换
cleaned_titles['production_countries'][1]

1    [US]
1    [US]
Name: production_countries, dtype: object

In [11]:
cleaned_titles = cleaned_titles.explode("production_countries")   # 拆分
cleaned_titles.head(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,thriller,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,european,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,fantasy,GB,NaN,tt0071853,8.2,534486.0,15.461,7.811
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,action,GB,NaN,tt0071853,8.2,534486.0,15.461,7.811
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,comedy,GB,NaN,tt0071853,8.2,534486.0,15.461,7.811


## **数据整洁度**

In [12]:
cleaned_titles.info()             #查看 cleaned_titles 基本信息，总行/列数，缺失值，数据类型

<class 'pandas.core.frame.DataFrame'>
Index: 17818 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    17818 non-null  object 
 1   title                 17817 non-null  object 
 2   type                  17818 non-null  object 
 3   description           17790 non-null  object 
 4   release_year          17818 non-null  int64  
 5   age_certification     10889 non-null  object 
 6   runtime               17818 non-null  int64  
 7   genres                17755 non-null  object 
 8   production_countries  17439 non-null  object 
 9   seasons               6224 non-null   float64
 10  imdb_id               17116 non-null  object 
 11  imdb_score            16976 non-null  float64
 12  imdb_votes            16945 non-null  float64
 13  tmdb_popularity       17663 non-null  float64
 14  tmdb_score            17241 non-null  float64
dtypes: float64(5), int64(2), 

**评估cleaned_titles的缺失数据，可以看出数据 imdb_score，genres有缺失值， release_year的数据类型应该为日期**

In [13]:
cleaned_titles["release_year"] = pd.to_datetime(cleaned_titles["release_year"], format='%Y')     #  release_year转换为日期类型
cleaned_titles["release_year"]

0      1945-01-01
1      1976-01-01
1      1976-01-01
2      1972-01-01
2      1972-01-01
          ...    
5847   2021-01-01
5848   2021-01-01
5849   2021-01-01
5849   2021-01-01
5849   2021-01-01
Name: release_year, Length: 17818, dtype: datetime64[ns]

In [14]:
cleaned_credits.info()           

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


**评估cleaned_credits的缺失数据，只有character存在缺失值。角色名并不影响我们挖掘各个流派中的高IMDB评分作品演员，此变量缺失也有可能因为演职员类别是导演，没有对应角色，因此可以允许character存在空缺的观察值。person_id数据一般不做数据处理，数据类型应该为字符串**

In [15]:
cleaned_credits["person_id"] = cleaned_credits["person_id"].astype("str")       # person_id数据类型转换为字符串
cleaned_credits["person_id"]

0           3748
1          14658
2           7064
3           3739
4          48933
          ...   
77796     736339
77797     399499
77798     373198
77799     378132
77800    1950416
Name: person_id, Length: 77801, dtype: object

**cleaned_titles数据缺失值处理**

In [16]:
cleaned_titles[cleaned_titles['imdb_score'].isnull()]     #查看 imdb_score缺失值 NaN，也可以写成 cleaned_titles.query("imdb_score.isnull()")

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945-01-01,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
75,tm132164,Bill Hicks: Sane Man,MOVIE,Sane Man was filmed before Bill recorded ‘Dang...,1989-01-01,R,80,comedy,US,NaN,NaN,NaN,NaN,3.377,7.5
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,documentation,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,family,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,reality,JP,12.0,NaN,NaN,NaN,7.730,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,2021-01-01,NaN,96,drama,NaN,NaN,tt20198164,NaN,NaN,NaN,NaN
5833,ts307884,HQ Barbers,SHOW,When a family run barber shop in the heart of ...,2021-01-01,TV-14,24,comedy,NG,1.0,NaN,NaN,NaN,0.840,NaN
5840,tm1216735,Sun of the Soil,MOVIE,"In 14th-century Mali, an ambitious young royal...",2022-01-01,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,1.179,7.0
5844,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",2021-01-01,NaN,35,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
cleaned_titles['imdb_score'].isnull().sum()               #查看 imdb_score 的缺失值个数

842

In [18]:
cleaned_titles.dropna(subset = ['imdb_score'], inplace = True)         #在原表格删除缺失值

        # inplace = True 在原表格更改，作用跟cleaned_titles = cleaned_titles.dropna()一样

In [19]:
cleaned_titles['imdb_score'].isnull().sum()               #查看 imdb_score 的缺失值删除后个数

0

In [20]:
cleaned_titles.query("genres.isnull()")            # 提取查看 genres的缺失值

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1813,ts77824,My Next Guest Needs No Introduction With David...,SHOW,TV legend David Letterman teams up with fascin...,2018-01-01,TV-MA,50,NaN,US,4.0,tt7829834,7.8,5581.0,8.217,7.6
1939,ts215037,Minecraft: Story Mode,SHOW,"MInecraft: Story Mode is an interactive, anima...",2018-01-01,TV-PG,52,NaN,US,1.0,tt10498322,5.6,347.0,NaN,NaN
2386,ts74805,A Little Help with Carol Burnett,SHOW,In this unscripted series starring comedy lege...,2018-01-01,TV-G,24,NaN,US,1.0,tt7204366,6.3,237.0,1.621,6.2
2658,ts265844,#ABtalks,SHOW,#ABtalks is a YouTube interview show hosted by...,2018-01-01,TV-PG,68,NaN,NaN,1.0,tt12635254,9.6,7.0,NaN,NaN
4274,tm1172010,The Lockdown Plan,MOVIE,NaN,2020-01-01,NaN,49,NaN,NaN,NaN,tt13079112,6.5,NaN,NaN,NaN
4648,tm1113921,In Vitro,MOVIE,'In Vitro' is an otherworldly rumination on me...,2019-01-01,NaN,27,NaN,NaN,NaN,tt10545994,7.7,NaN,NaN,NaN


In [21]:
cleaned_titles = cleaned_titles.dropna(subset=["genres"])       # 删除 genres缺失值并验证结果
cleaned_titles["genres"].isnull().sum()

0

## **处理重复数据**

In [22]:
cleaned_titles['id'].duplicated().sum()                   # 根据 cleaned_titles 内容含义，查看 id 是否有重复值

11608

In [23]:
cleaned_credits.duplicated().sum()                    # 查看 cleaned_credits 整体是否有重复值

0

## **不一致数据**

**查看是否存在多个不同值指代同一流派，以及多个不同值指代同一国家的情况**

In [24]:
cleaned_titles['genres'].value_counts()          # genres流派计数

genres
drama            3357
comedy           2419
thriller         1446
action           1339
romance          1080
crime            1066
documentation     981
family            769
animation         732
fantasy           727
european          679
scifi             647
horror            438
history           336
music             266
reality           226
war               221
sport             188
western            53
Name: count, dtype: int64

**从上面看出，genres列里并不存在不一致数据，各个值都在指代不同的流派。也没有空字符串 ''**

In [25]:
cleaned_titles.query('genres == ""')                          # 查看 genres为空字符串的行
# cleaned_titles = cleaned_titles.query('genres !== ""') 若有空字符串''，可以用这样删除 

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


In [26]:
cleaned_titles['production_countries'].value_counts()

production_countries
US    5648
IN    1610
GB    1068
JP    1046
FR     720
      ... 
GT       1
CU       1
LK       1
NP       1
FO       1
Name: count, Length: 108, dtype: int64

**由于`value_counts`执行结果中值比较多，Pandas默认只显示开头和结尾的一些值。要完整展示结果，可以把`display.max_rows`设置为`None`，取消展示行数上限。
因为我们只是在当前调用`value_counts`时才需要看完整结果，所以可以结合`option_context`，更改临时上限：**

In [27]:
with pd.option_context('display.max_rows', None):
    print(cleaned_titles['production_countries'].value_counts())

production_countries
US         5648
IN         1610
GB         1068
JP         1046
FR          720
KR          637
ES          637
CA          608
DE          383
CN          295
MX          264
IT          224
BR          221
AU          217
TR          195
PH          192
AR          150
ID          149
BE          148
TW          133
NG          131
PL          126
ZA          103
NL          102
HK          102
CO           94
EG           93
DK           89
TH           87
SE           81
LB           70
NO           68
AE           52
IE           49
SG           47
XX           43
IL           42
RU           41
CL           35
CH           33
PS           32
BG           31
MY           30
SA           28
AT           28
IS           28
LU           27
NZ           27
PE           26
RO           25
QA           24
CZ           22
JO           19
FI           18
HU           18
UY           15
MA           15
PT           14
KH           10
KW           10
PR            9
PK 

**从以上输出结果来看，出品国家都用两位的国家代码来表示，除了里面存在一个的`Lebanon`值。
`Lebanon`的国家代码是`LB`，出现了39次，说明此处数据不一致。`LB`和`Lebanon`都在表示同一国家，需要进行统一。**

In [28]:
# 对 "production_countries"列运用 replace函数把 Lebanon 替换为 LB
cleaned_titles["production_countries"] = cleaned_titles["production_countries"].replace({"Lebanon": "LB"})

# 再次查看
with pd.option_context('display.max_rows', None):
    print(cleaned_titles.explode('production_countries')['production_countries'].value_counts())

production_countries
US    5648
IN    1610
GB    1068
JP    1046
FR     720
KR     637
ES     637
CA     608
DE     383
CN     295
MX     264
IT     224
BR     221
AU     217
TR     195
PH     192
AR     150
ID     149
BE     148
TW     133
NG     131
PL     126
ZA     103
HK     102
NL     102
CO      94
EG      93
DK      89
TH      87
SE      81
LB      71
NO      68
AE      52
IE      49
SG      47
XX      43
IL      42
RU      41
CL      35
CH      33
PS      32
BG      31
MY      30
AT      28
SA      28
IS      28
LU      27
NZ      27
PE      26
RO      25
QA      24
CZ      22
JO      19
FI      18
HU      18
UY      15
MA      15
PT      14
KW      10
KH      10
PK       9
PR       9
UA       8
MT       8
VN       8
LT       7
IR       7
CD       7
SU       7
TN       7
SN       6
AL       6
KE       6
GH       6
IQ       5
MU       5
CY       5
KN       4
GR       4
IO       4
SY       4
TZ       4
MC       4
GL       3
CM       3
AO       3
BS       3
HR       3
BD       3


In [29]:
original_credits['role'].value_counts()           # 结果上看 role没有不一致的值

role
ACTOR       73251
DIRECTOR     4550
Name: count, dtype: int64

**从以上输出结果来看，`role`只有两种可能的值，`ACTOR`或`DIRECTOR`，不存在不一致数据。可以把这列的类型转换为`Category`（类别），`Category`用于固定的且有限数量的变量中，比字符串类型更节约内存空间。**

In [30]:
cleaned_credits["role"] = cleaned_credits["role"].astype("category")
cleaned_credits["role"]

0           ACTOR
1           ACTOR
2           ACTOR
3           ACTOR
4           ACTOR
           ...   
77796       ACTOR
77797       ACTOR
77798       ACTOR
77799       ACTOR
77800    DIRECTOR
Name: role, Length: 77801, dtype: category
Categories (2, object): ['ACTOR', 'DIRECTOR']

## **无效或错误数据**

In [31]:
cleaned_titles.describe()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,16970,16970.000000,5954.000000,16970.000000,1.694100e+04,16842.000000,16515.000000
mean,2015-11-14 22:42:51.974072064,80.912552,2.455492,6.514207,3.281655e+04,29.396307,6.846933
min,1954-01-01 00:00:00,0.000000,1.000000,1.500000,5.000000e+00,0.600000,1.000000
25%,2015-01-01 00:00:00,45.000000,1.000000,5.800000,7.800000e+02,4.070000,6.200000
50%,2018-01-01 00:00:00,90.000000,2.000000,6.600000,3.508000e+03,10.195000,6.900000
75%,2020-01-01 00:00:00,107.000000,3.000000,7.300000,1.697800e+04,23.639000,7.500000
max,2022-01-01 00:00:00,225.000000,42.000000,9.500000,2.294231e+06,2274.044000,10.000000
std,NaN,39.596172,2.869428,1.131095,1.141492e+05,93.178235,1.078831


- **cleaned_titles从结果观察没有脱离实际意义的值**
- **cleaned_credits 不存在数值含义的数据，不用describe()**

## **整理数据2.0**

**两个表格都有id列，意思相同，通过关联 id 进行合并**

In [32]:
credits_with_titles = pd.merge(cleaned_credits, cleaned_titles, on="id", how="inner")
                       # pd.merge，（表格）合并       ps:数组是 concatenate 拼接

In [33]:
credits_with_titles.head() 

,person_id,id,name,character,role,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
3,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
4,7064,tm84618,Albert Brooks,Tom,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179


**由于是挖掘演员的参演作品口碑，导演不在我们的分析范围内，根据role，筛选出类型为ACTOR的观察值：**

In [34]:
actor_with_titles = credits_with_titles.query('role == "ACTOR"')

**根据流派和演员进行分组。对演员进行分组时，用person_id而不是name，原因是名字有重名和错拼的可能，演员ID会比演员姓名更加准确地反映是哪位演员：**

In [35]:
groupby_genres_and_person_id = actor_with_titles.groupby(["genres", "person_id"])

**分组后，提取imdb_score变量，然后调用mean，来计算各个流派影视作品中，每位演员参演作品的平均IMDB评分：**

In [36]:
imdb_score_groupby_genres_and_person_id = groupby_genres_and_person_id["imdb_score"].mean()
imdb_score_groupby_genres_and_person_id

genres   person_id
action   1000         6.866667
         100007       7.000000
         100013       6.400000
         100019       6.500000
         100020       6.500000
                        ...   
western  993735       6.500000
         998673       7.300000
         998674       7.300000
         998675       7.300000
         99940        4.000000
Name: imdb_score, Length: 168881, dtype: float64

**调用`reset_index`，对层次化索引进行重置，得到更加规整的DataFrame：**

In [37]:
imdb_score_groupby_genres_and_person_id_df = imdb_score_groupby_genres_and_person_id.reset_index()
imdb_score_groupby_genres_and_person_id_df

,genres,person_id,imdb_score
0,action,1000,6.866667
1,action,100007,7.000000
2,action,100013,6.400000
3,action,100019,6.500000
4,action,100020,6.500000
...,...,...,...
168876,western,993735,6.500000
168877,western,998673,7.300000
168878,western,998674,7.300000
168879,western,998675,7.300000


**流派和演员分组的IMDB评分数据整理好后，可以对结果再次进行分组，找出各个流派里演员作品最高的平均评分是多少、最高评分对应的演员名字是什么。再次用`genres`进行分组，然后提取出`imdb_score`变量，计算其最大值。**

In [38]:
genres_max_scores = imdb_score_groupby_genres_and_person_id_df.groupby("genres")["imdb_score"].max()
genres_max_scores

genres
action           9.3
animation        9.3
comedy           9.2
crime            9.5
documentation    9.1
drama            9.5
european         8.9
family           9.3
fantasy          9.3
history          9.1
horror           9.0
music            8.8
reality          8.9
romance          9.2
scifi            9.3
sport            9.1
thriller         9.5
war              8.8
western          8.9
Name: imdb_score, dtype: float64

**以上结果和之前得到的`imdb_score_groupby_genres_and_person_id_df`再次进行连接，得到最高分对应的各个演员ID**

In [39]:
genres_max_score_with_person_id = pd.merge(imdb_score_groupby_genres_and_person_id_df, genres_max_scores, on=["genres", "imdb_score"])
genres_max_score_with_person_id

,genres,person_id,imdb_score
0,action,12790,9.3
1,action,1303,9.3
2,action,21033,9.3
3,action,336830,9.3
4,action,86591,9.3
...,...,...,...
131,war,826547,8.8
132,western,22311,8.9
133,western,28166,8.9
134,western,28180,8.9


**从以上结果可以看出，最高分对应的演员不一定只有一位。cleaned_credits和DataFrame进行连接，得到演员ID所对应的演员名字。这个DataFrame还有其它列，我们只需要得到person_id和name的对应，可以先提取出这两列，并把重复行删除。**

In [40]:
actor_id_with_names = cleaned_credits[['person_id', 'name']].drop_duplicates()
actor_id_with_names.head(10)

,person_id,name
0,3748,Robert De Niro
1,14658,Jodie Foster
2,7064,Albert Brooks
3,3739,Harvey Keitel
4,48933,Cybill Shepherd
5,32267,Peter Boyle
6,519612,Leonard Harris
7,29068,Diahnne Abbott
8,519613,Gino Ardito
9,3308,Martin Scorsese


**把`actor_id_with_names`与前面得到的`genres_max_score_with_person_id`进行连接，增加`name`变量，从而展示平均评分最高的演员名字。**

In [41]:
genres_max_score_with_actor_name = pd.merge(genres_max_score_with_person_id, actor_id_with_names, on="person_id")
genres_max_score_with_actor_name

,genres,person_id,imdb_score,name
0,action,12790,9.3,Olivia Hack
1,scifi,12790,9.3,Olivia Hack
2,action,1303,9.3,Jessie Flower
3,animation,1303,9.3,Jessie Flower
4,family,1303,9.3,Jessie Flower
...,...,...,...,...
131,war,826547,8.8,Yuto Uemura
132,western,22311,8.9,Koichi Yamadera
133,western,28166,8.9,Megumi Hayashibara
134,western,28180,8.9,Unsho Ishizuka


**用sort_values方法把相同流派都排序在一起，然后把结果里面的行根据genres进行排序，用reset_index把索引重新排序。索引重新排序后，DataFrame会多出index一列，可以对index列进行删除。**

In [42]:
genres_max_score_with_actor_name = genres_max_score_with_actor_name.sort_values("genres").reset_index().drop("index", axis=1)
genres_max_score_with_actor_name

,genres,person_id,imdb_score,name
0,action,12790,9.3,Olivia Hack
1,action,336830,9.3,André Sogliuzzo
2,action,21033,9.3,Zach Tyler
3,action,86591,9.3,Cricket Leigh
4,action,1303,9.3,Jessie Flower
...,...,...,...,...
131,war,826547,8.8,Yuto Uemura
132,western,28166,8.9,Megumi Hayashibara
133,western,28180,8.9,Unsho Ishizuka
134,western,22311,8.9,Koichi Yamadera


In [43]:
genres_max_score_with_actor_name[genres_max_score_with_actor_name['genres'] == 'scifi']    # 提取查看指定流派

,genres,person_id,imdb_score,name
115,scifi,12790,9.3,Olivia Hack
116,scifi,86591,9.3,Cricket Leigh
117,scifi,28024,9.3,Dante Basco
118,scifi,336830,9.3,André Sogliuzzo
119,scifi,21033,9.3,Zach Tyler
120,scifi,1303,9.3,Jessie Flower
